In [1]:
import time
import numpy as np
import torch

In [2]:
def recursive_count(n):
    if n == 0:
        return 0
    return 1+recursive_count(n-1)

In [3]:
'''
loop = list()
for _ in range(10000):
    stime = time.time()
    recursive_count(2000)
    recursive_count(2000)
    recursive_count(2000)
    recursive_count(2000)
    recursive_count(2000)
    recursive_count(2000)
    recursive_count(2000)
    recursive_count(2000)
    recursive_count(2000)
    recursive_count(2000)
    etime = time.time()
    loop.append(etime - stime)

print(np.mean(loop))
'''

'\nloop = list()\nfor _ in range(10000):\n    stime = time.time()\n    recursive_count(2000)\n    recursive_count(2000)\n    recursive_count(2000)\n    recursive_count(2000)\n    recursive_count(2000)\n    recursive_count(2000)\n    recursive_count(2000)\n    recursive_count(2000)\n    recursive_count(2000)\n    recursive_count(2000)\n    etime = time.time()\n    loop.append(etime - stime)\n\nprint(np.mean(loop))\n'

In [4]:
'''
lsloop = list()
for _ in range(10000):
    stime = time.time()
    for i in range(10):
        recursive_count(2000)
    etime = time.time()
    lsloop.append(etime - stime)

print(np.mean(lsloop))
'''

'\nlsloop = list()\nfor _ in range(10000):\n    stime = time.time()\n    for i in range(10):\n        recursive_count(2000)\n    etime = time.time()\n    lsloop.append(etime - stime)\n\nprint(np.mean(lsloop))\n'

In [5]:
from Net import Net
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    torch.set_default_tensor_type(torch.cuda.FloatTensor)


In [6]:
net = Net(3,[10,10,10,10])
input = torch.tensor([[1,1.,1,1]])
net(input)

tensor([[-0.3673]], grad_fn=<MulBackward0>)

In [7]:
from PINN import PINN

model = PINN([10,10,10,10])

In [8]:
model.loss_function(input,1,1,1)

torch.Size([1, 1]) torch.Size([1, 3])


(tensor(0.0090, grad_fn=<AddBackward0>),
 tensor(0.0090, grad_fn=<MeanBackward0>),
 tensor(0., grad_fn=<MeanBackward0>))

In [9]:
x = torch.tensor([[2,2,2,2.],[1,1,1,1.]],requires_grad=True)
y = torch.tensor([[1,2,3,4.],[5,6,7,8.]],requires_grad=True)
u = x*3
torch.autograd.grad(outputs=u,inputs=x,grad_outputs=torch.ones_like(u), create_graph=True, retain_graph=True)

(tensor([[3., 3., 3., 3.],
         [3., 3., 3., 3.]]),)

In [10]:
x*y

tensor([[2., 4., 6., 8.],
        [5., 6., 7., 8.]], grad_fn=<MulBackward0>)

In [11]:
import matplotlib.pyplot as plt

# load the best model 
Z_value = .5
net.load_state_dict(torch.load('best_model.pth'))

# Define the grid and time steps
n= 100
x_grid = np.linspace(0, 1, n)
y_grid = np.linspace(0, 1, n)
z_grid = np.linspace(0, 1, n)

X, Y, = np.meshgrid(x_grid, y_grid)
Z= X *0+ Z_value

grid_points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T
# grid_points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T  # Flatten the grid
grid_points = torch.tensor(grid_points, dtype=torch.float32)

time_steps = np.linspace(0, t_final, 100)  # 10 time steps from 0 to 1


'''
# Plot the concentration over time
for t in time_steps:
    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input
    concentration = net(grid_points, t_tensor).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape
    
    plt.figure()
    plt.contourf(X, Y, concentration, levels=50, cmap='viridis', vmin=0, vmax=10)  # Plot concentration as a contour plot
    plt.plot(source_loc[0,0].cpu(), source_loc[0,1].cpu(), 'ro', label='Source Location')  # Plot the source location
    plt.colorbar(label='Concentration')
    # fix colorbar from 0 to 1 
    # plt.clim(0, 10.0)  # Set colorbar limits
    plt.title(f"Gas Concentration at t = {t:.2f}")
    plt.xlabel('x')
    plt.ylabel('y')
    # fix colorbar 
    # plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image
'''

FileNotFoundError: [Errno 2] No such file or directory: 'best_model.pth'

In [ ]:
# save as a GIF
import imageio
import os

images = []

for t in time_steps:
    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input
    concentration = net(grid_points, t_tensor).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape
    
    plt.figure()
    plt.contourf(X, Y, concentration, levels=50, cmap='viridis', vmin=0, vmax=.5)  # Plot concentration as a contour plot
    plt.plot(source_loc[0,0].cpu(), source_loc[0,1].cpu(), 'ro', label='Source Location')  # Plot the source location
    plt.colorbar(label='Concentration')
    plt.title(f"Gas Concentration at t = {t:.2f}")
    plt.xlabel('x')
    plt.ylabel('y')
    plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image
    plt.close()
    
    images.append(imageio.imread(f"concentration_t_{t:.2f}.png"))  # Append the image to the list
    os.remove(f"concentration_t_{t:.2f}.png")  # Remove the image file

imageio.mimsave(f'concentration_gaussian_source_3d_Z_{round(Z_value,2)}.gif', images)  # Save the images as a GIF

